In [80]:
import numpy as np
import pandas as pd
from plotly import graph_objects as go


In [5]:
def leastsqr(x, y):
    b = (np.mean(x * y) - np.mean(x) * np.mean(y)) /\
        (np.mean(x * x) - np.mean(x) ** 2)
    a = np.mean(y) - b * np.mean(x)
    sig_b = np.sqrt(((np.mean(y ** 2) - np.mean(y) ** 2) /
                    ((np.mean(x ** 2) - np.mean(x) ** 2)) - b ** 2)/len(x))
    sig_a = sig_b * (np.mean(x * x) - np.mean(x) ** 2) ** 0.5
    return b, a, sig_b, sig_a



# Результаты измерений

In [83]:
t = [1090, 1150, 1250, 1324, 1386, 1450, 1564, 1722, 1670, 1703, 1500, 1800, 1900, 1960]
a = [0.596, 0.641, 0.668, 0.673, 0.707, 0.740, 0.833, 0.909, 0.855, 0.883, 0.813, 0.927, 1.010, 1.07]
v = [2.47, 2.935, 3.222, 3.28, 3.658, 4.03, 5.185, 6.190, 5.465, 5.830, 4.930, 6.438, 7.635, 8.560]
a = np.array(a)
v = np.array(v)
y = np.sort(a * v)
x = np.sort(t)
data = pd.DataFrame([a, v, t]).T
data.columns= ['Напряжение на нити (В)', 'Сила тока на нити (А)', 'Температура (°C)']
data

,Напряжение на нити (В),Сила тока на нити (А),Температура (°C)
0,0.596,2.470,1090.0
1,0.641,2.935,1150.0
2,0.668,3.222,1250.0
3,0.673,3.280,1324.0
4,0.707,3.658,1386.0
5,0.740,4.030,1450.0
6,0.833,5.185,1564.0
7,0.909,6.190,1722.0
8,0.855,5.465,1670.0
9,0.883,5.830,1703.0


## Зависимость температуры от энергии
В теории должна описоваться согласно формуле
$$
W=\varepsilon_{T} B T^{n},
$$
где $n=4$

In [85]:
cur_trace = go.Scatter({
        'x': x,
        'y': y,
        
        'marker': {
            'color': 'red'
        },
        'name': '{} час'.format(t),
        
        'line_shape': 'spline',
        'mode': 'lines+markers'
    })

layout = go.Layout({
    'title': 'Результаты',
    'height': 500,
    'width': 700,
    'xaxis': {
        'title': 'Температура (°C)',
    },
    'yaxis': {
        'title': 'Мощность (Вт)'
    }
})

go.Figure(data=[cur_trace], layout=layout)   

Прологарифмировав обе части выражения (1) получим:
$$
\ln W=\ln \left(\varepsilon_{T} B\right)+n \ln T
$$
Тогда коэффициент n может быть найден из метода наименьших квадратов как коэффициент наклона прямой

In [87]:
points = go.Scatter({
    'x': np.log(x),
    'y': np.log(y),
    'name': 'Измерения',  
    'error_y': {
        'type': 'constant',
        'value':0.1,
    },
    'mode': 'markers'
    })

x_linspace = np.linspace(min(np.log(x)), max(np.log(x)), 200)

n_1, ln_eps, sig_n1, sig_ln_eps = leastsqr(np.log(x), np.log(y))

linear_approx = go.Scatter({
    'x': x_linspace,
    'y': n_1 * x_linspace + ln_eps,
    'name': 'Линейная апроксимация',
    
})

layout = go.Layout({
    'title': 'Логарифмическая зависимость мощности от температуры',
    'height': 500,
    'width': 800,
    'xaxis': {
        'title': 'log(T)'
    },
    'yaxis': {
        'title': 'log(W)'
    }
})
print('Коэффициент наклона прямой: {0:.2f}±{1:.2f}'.format(n_1, sig_n1))
go.Figure(data=[points, linear_approx], layout=layout) 

Коэффициент наклона прямой: 2.99±0.12


Если выкинуть все значения меньше 1700K получим:

In [88]:

x_ = x[-6:]
y_ = y[-6:]

points = go.Scatter({
    'x': np.log(x_),
    'y': np.log(y_),
    'name': 'Измерения',  
    'error_y': {
        'type': 'constant',
        'value':0.1,
    },
    'mode': 'markers'
    })

x_linspace = np.linspace(min(np.log(x_)), max(np.log(x_)), 200)

n_1, ln_eps, sig_n1, sig_ln_eps = leastsqr(np.log(x_), np.log(y_))

linear_approx = go.Scatter({
    'x': x_linspace,
    'y': n_1 * x_linspace + ln_eps,
    'name': 'Линейная апроксимация',
    
})

layout = go.Layout({
    'title': 'Логарифмическая зависимость мощности от температуры',
    'height': 500,
    'width': 800,
    'xaxis': {
        'title': 'log(T)'
    },
    'yaxis': {
        'title': 'log(W)'
    }
})
print('Коэффициент наклона прямой: {0:.2f}±{1:.2f}'.format(n_1, sig_n1))
go.Figure(data=[points, linear_approx], layout=layout)

Коэффициент наклона прямой: 3.92±0.26


## Вычисление коэффициента постоянной Больцмана

Ясно, что в формуле (2) 
$$
B=S \cdot \sigma,
$$
где $S$ – эффективная площадь излучающей лампы при температуре более 1500°C, $\sigma$ – постоянная Стефана-Больцмана; $S = 0,36 \text{см}^2 $. Тогда:
$$
\sigma=\frac{W}{\varepsilon_{T} S T^{4}}.
$$
Найдем ее значение для каждого измеренного значения $T$, превышающего 1700K.
Будем считать, что значения коэффициента $\varepsilon_t$ линейно зависит от температуры. Тогда:

In [75]:
S = 0.36 * 10 ** -4
x_k = x_ + 273
k_1, k_2, sig_k_1, sig_k_2 = leastsqr(np.array([1700, 1800, 1900]), np.array([0.209, 0.223, 0.236]))
eps_t = k_1 * (x_k) + k_2
sigma = y_ / (eps_t * S * x_k ** 4)
sigma

array([3.76355322e-08, 3.80619728e-08, 3.96274037e-08, 3.45905427e-08,
       3.51877822e-08, 3.64003727e-08])

Реальное зачение $\sigma=5,670367(13) \times 10^{-8} \text{Вт}  \cdot \text{м} ^{-2} \cdot \mathrm{K}^{-4}$

# Вычисление постоянной планка
Постоянная планка может быть вычислена по формуле:
$$
h=\sqrt[3]{\frac{2 \pi^{5} k_{\mathrm{\text{Б}}}^{4}}{15 c^{2} \sigma}}
$$

In [79]:
k_b = 1.38064852 * 10 ** -23
c = 3 * 10 ** 8
h = ((2 * np.pi ** 5 * k_b ** 4) / (15 * c **2 * sigma)) ** (1/3)
h

array([7.59264950e-34, 7.56418731e-34, 7.46324158e-34, 7.80920662e-34,
       7.76477258e-34, 7.67757550e-34])

Реальное значение $h = 6,62607004 \times 10^{-34} \text{Дж} \cdot \text{К} $